<a href="https://colab.research.google.com/github/saddarudin/google_colab/blob/main/nlp_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df.Category.value_counts()

,count
Category,
ham,4825
spam,747


In [6]:
df['spam'] = df.Category.apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [7]:
df.drop('Category',axis='columns',inplace=True)
df.head()

,Message,spam
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.Message,df.spam,test_size=0.2)
x_train.shape,x_test.shape

((4457,), (1115,))

In [12]:
y_train.value_counts()

,count
spam,
0,3865
1,592


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train_cv = cv.fit_transform(x_train.values)
x_test_cv = cv.transform(x_test.values)

In [15]:
x_train_cv.shape

(4457, 7755)

In [16]:
x_train_cv.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
cv.vocabulary_

{'for': 2973,
 'ur': 7216,
 'chance': 1743,
 'to': 6954,
 'win': 7537,
 '250': 370,
 'cash': 1698,
 'every': 2698,
 'wk': 7571,
 'txt': 7118,
 'action': 805,
 '80608': 652,
 'www': 7652,
 'movietrivia': 4632,
 'tv': 7107,
 'custcare': 2159,
 '08712405022': 113,
 '1x150p': 347,
 'customer': 2161,
 'loyalty': 4265,
 'offer': 4918,
 'the': 6839,
 'new': 4780,
 'nokia6650': 4826,
 'mobile': 4568,
 'from': 3049,
 'only': 4966,
 '10': 261,
 'at': 1132,
 'txtauction': 7121,
 'word': 7604,
 'start': 6471,
 'no': 4814,
 '81151': 656,
 'get': 3144,
 'yours': 7727,
 'now': 4864,
 '4t': 526,
 'ctxt': 2137,
 'tc': 6754,
 '150p': 313,
 'mtmsg': 4652,
 'plane': 5257,
 'give': 3167,
 'on': 4955,
 'this': 6876,
 'month': 4600,
 'end': 2621,
 'how': 3552,
 'much': 4658,
 'an': 967,
 'eighth': 2588,
 'don': 2443,
 'make': 4350,
 'life': 4128,
 'too': 6995,
 'stressfull': 6543,
 'always': 940,
 'find': 2886,
 'time': 6923,
 'laugh': 4060,
 'it': 3793,
 'may': 4421,
 'not': 4850,
 'add': 813,
 'years': 769

In [21]:
cv.get_feature_names_out()[1743]

'chance'

In [22]:
x_train_np = x_train_cv.toarray()
x_train_np[:4]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_cv,y_train)

MultinomialNB()

In [24]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test_cv)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       960
           1       0.97      0.96      0.96       155

    accuracy                           0.99      1115
   macro avg       0.98      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [25]:
emails = [
    "Hey, Mohan can we get together to watch football game tomorrow?",
    "Upto 20% discount on parking, exclusive offer just for you. Dont' miss this reward!"
]

emails_count = cv.transform(emails)
model.predict(emails_count)

array([0, 1])

## Doing same thing using pipeline

In [26]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [27]:
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [28]:
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       960
           1       0.97      0.96      0.96       155

    accuracy                           0.99      1115
   macro avg       0.98      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115

